<a href="https://colab.research.google.com/github/Python-Neiva/risk-factors-covid19/blob/main/ResearchInPython.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Model

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
from math import*

In [2]:
import os
!pip install sodapy -q
!pip install socrata-py -q

from sodapy import Socrata
from socrata.authorization import Authorization

In [3]:
client = Socrata(
                "www.datos.gov.co",
                "idLwSviPi5dGMTd7GfIfW3CLU",
                #username="sebastian_romerola@fet.edu.co",
                #password="Sebastian#2018",
                timeout=120
                )

results = client.get("gt2j-8ykr",
                    #  ciudad_municipio_nom="Neiva",
                    #  departamento_nom="HUILA",
                    limit=2000
                    )

In [4]:
df = pd.DataFrame.from_records(results)

In [5]:
#  path= 'https://www.datos.gov.co/resource/gt2j-8ykr.csv'
#
#  df = pd.read_csv (r''+path)
#  df.head(10)

In [6]:
df.head(10)

,fecha_reporte_web,id_de_caso,fecha_de_notificaci_n,departamento,departamento_nom,ciudad_municipio,ciudad_municipio_nom,edad,unidad_medida,sexo,fuente_tipo_contagio,ubicacion,estado,pais_viajo_1_cod,pais_viajo_1_nom,recuperado,fecha_inicio_sintomas,fecha_diagnostico,fecha_recuperado,tipo_recuperacion,per_etn_,nom_grupo_,fecha_muerte
0,6/3/2020 0:00:00,1,2/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,19,1,F,Importado,Casa,Leve,380,ITALIA,Recuperado,27/2/2020 0:00:00,6/3/2020 0:00:00,13/3/2020 0:00:00,PCR,6,NaN,NaN
1,9/3/2020 0:00:00,2,6/3/2020 0:00:00,76,VALLE,76111,BUGA,34,1,M,Importado,Casa,Leve,724,ESPAÑA,Recuperado,4/3/2020 0:00:00,9/3/2020 0:00:00,19/3/2020 0:00:00,PCR,5,NaN,NaN
2,9/3/2020 0:00:00,3,7/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,50,1,F,Importado,Casa,Leve,724,ESPAÑA,Recuperado,29/2/2020 0:00:00,9/3/2020 0:00:00,15/3/2020 0:00:00,PCR,6,NaN,NaN
3,11/3/2020 0:00:00,4,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,55,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,6/3/2020 0:00:00,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6,NaN,NaN
4,11/3/2020 0:00:00,5,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,25,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,8/3/2020 0:00:00,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6,NaN,NaN
5,11/3/2020 0:00:00,6,10/3/2020 0:00:00,5,ANTIOQUIA,5360,ITAGUI,27,1,F,Relacionado,Casa,Leve,NaN,NaN,Recuperado,6/3/2020 0:00:00,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6,NaN,NaN
6,11/3/2020 0:00:00,7,8/3/2020 0:00:00,13001,CARTAGENA,13001,CARTAGENA,85,1,F,Importado,Casa,Leve,840,ESTADOS UNIDOS DE AMÉRICA,Recuperado,2/3/2020 0:00:00,11/3/2020 0:00:00,17/3/2020 0:00:00,PCR,6,NaN,NaN
7,11/3/2020 0:00:00,8,9/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,22,1,F,Importado,Casa,Leve,724,ESPAÑA,Recuperado,6/3/2020 0:00:00,11/3/2020 0:00:00,21/3/2020 0:00:00,PCR,6,NaN,NaN
8,11/3/2020 0:00:00,9,8/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,28,1,F,Importado,Casa,Leve,724,ESPAÑA,Recuperado,7/3/2020 0:00:00,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6,NaN,NaN
9,12/3/2020 0:00:00,10,12/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,36,1,F,Importado,Casa,Leve,724,ESPAÑA,Recuperado,6/3/2020 0:00:00,12/3/2020 0:00:00,21/3/2020 0:00:00,PCR,6,NaN,NaN


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   fecha_reporte_web      2000 non-null   object
 1   id_de_caso             2000 non-null   object
 2   fecha_de_notificaci_n  2000 non-null   object
 3   departamento           2000 non-null   object
 4   departamento_nom       2000 non-null   object
 5   ciudad_municipio       2000 non-null   object
 6   ciudad_municipio_nom   2000 non-null   object
 7   edad                   2000 non-null   object
 8   unidad_medida          2000 non-null   object
 9   sexo                   2000 non-null   object
 10  fuente_tipo_contagio   2000 non-null   object
 11  ubicacion              2000 non-null   object
 12  estado                 2000 non-null   object
 13  pais_viajo_1_cod       684 non-null    object
 14  pais_viajo_1_nom       684 non-null    object
 15  recuperado           

In [8]:
df.describe()

,fecha_reporte_web,id_de_caso,fecha_de_notificaci_n,departamento,departamento_nom,ciudad_municipio,ciudad_municipio_nom,edad,unidad_medida,sexo,fuente_tipo_contagio,ubicacion,estado,pais_viajo_1_cod,pais_viajo_1_nom,recuperado,fecha_inicio_sintomas,fecha_diagnostico,fecha_recuperado,tipo_recuperacion,per_etn_,nom_grupo_,fecha_muerte
count,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,684,684,2000,1938,1998,1870,1870,2000,18,137
unique,31,2000,37,27,27,117,117,96,2,2,4,3,3,37,37,3,40,31,114,2,4,7,51
top,8/4/2020 0:00:00,444,28/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,29,1,M,Relacionado,Casa,Leve,724,ESPAÑA,Recuperado,20/3/2020 0:00:00,8/4/2020 0:00:00,5/5/2020 0:00:00,PCR,6,Por definir,4/4/2020 0:00:00
freq,292,1,143,975,975,975,975,58,1995,1004,1004,1863,1863,236,236,1870,141,287,116,1695,1937,9,12


In [9]:
np.shape(df)

(2000, 23)

In [10]:
df.columns

Index(['fecha_reporte_web', 'id_de_caso', 'fecha_de_notificaci_n',
       'departamento', 'departamento_nom', 'ciudad_municipio',
       'ciudad_municipio_nom', 'edad', 'unidad_medida', 'sexo',
       'fuente_tipo_contagio', 'ubicacion', 'estado', 'pais_viajo_1_cod',
       'pais_viajo_1_nom', 'recuperado', 'fecha_inicio_sintomas',
       'fecha_diagnostico', 'fecha_recuperado', 'tipo_recuperacion',
       'per_etn_', 'nom_grupo_', 'fecha_muerte'],
      dtype='object')

In [11]:
headers_name = [
  'fecha_reporte_web',
  'id_de_caso',
  'fecha_de_notificaci_n',
  'departamento',
  'departamento_nom',
  'ciudad_municipio',
  'ciudad_municipio_nom',
  'edad',
  'unidad_medida',
  'sexo',
  'fuente_tipo_contagio',
  'ubicacion',
  'estado',
  'pais_viajo_1_cod',
  'pais_viajo_1_nom',
  'recuperado',
  'fecha_inicio_sintomas',
  'fecha_diagnostico',
  'fecha_recuperado',
  'tipo_recuperacion',
  'per_etn_',
  'fecha_muerte',
  'nom_grupo_',
]

In [12]:
df.columns = [
  'web_date_report',
  'case_id',
  'notification_date',
  'davipola_departament_code',
  'department_name',
  'davipola_municipality_code',
  'municipality_name',
  'age',
  'age_measurement_unit',
  'sex',
  'contagion_type',
  'case_location',
  'state',
  'iso_country_code',
  'country_name',
  'recuperated', #spell correctly
  'beginning_symptoms_date',#!/usr/bin/env python
  'date_of_diagnosis',
  'date_of_recovery',
  'recovery_type',
  'ethnicity',
  'ethnic_group_name',
  'date_of_death',
]

#df.columns[2,3,5,8,9,10,11,12,13,15,16,17,18,20,21]

In [13]:
df.head(10)

,web_date_report,case_id,notification_date,davipola_departament_code,department_name,davipola_municipality_code,municipality_name,age,age_measurement_unit,sex,contagion_type,case_location,state,iso_country_code,country_name,recuperated,beginning_symptoms_date,date_of_diagnosis,date_of_recovery,recovery_type,ethnicity,ethnic_group_name,date_of_death
0,6/3/2020 0:00:00,1,2/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,19,1,F,Importado,Casa,Leve,380,ITALIA,Recuperado,27/2/2020 0:00:00,6/3/2020 0:00:00,13/3/2020 0:00:00,PCR,6,NaN,NaN
1,9/3/2020 0:00:00,2,6/3/2020 0:00:00,76,VALLE,76111,BUGA,34,1,M,Importado,Casa,Leve,724,ESPAÑA,Recuperado,4/3/2020 0:00:00,9/3/2020 0:00:00,19/3/2020 0:00:00,PCR,5,NaN,NaN
2,9/3/2020 0:00:00,3,7/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,50,1,F,Importado,Casa,Leve,724,ESPAÑA,Recuperado,29/2/2020 0:00:00,9/3/2020 0:00:00,15/3/2020 0:00:00,PCR,6,NaN,NaN
3,11/3/2020 0:00:00,4,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,55,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,6/3/2020 0:00:00,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6,NaN,NaN
4,11/3/2020 0:00:00,5,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,25,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,8/3/2020 0:00:00,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6,NaN,NaN
5,11/3/2020 0:00:00,6,10/3/2020 0:00:00,5,ANTIOQUIA,5360,ITAGUI,27,1,F,Relacionado,Casa,Leve,NaN,NaN,Recuperado,6/3/2020 0:00:00,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6,NaN,NaN
6,11/3/2020 0:00:00,7,8/3/2020 0:00:00,13001,CARTAGENA,13001,CARTAGENA,85,1,F,Importado,Casa,Leve,840,ESTADOS UNIDOS DE AMÉRICA,Recuperado,2/3/2020 0:00:00,11/3/2020 0:00:00,17/3/2020 0:00:00,PCR,6,NaN,NaN
7,11/3/2020 0:00:00,8,9/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,22,1,F,Importado,Casa,Leve,724,ESPAÑA,Recuperado,6/3/2020 0:00:00,11/3/2020 0:00:00,21/3/2020 0:00:00,PCR,6,NaN,NaN
8,11/3/2020 0:00:00,9,8/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,28,1,F,Importado,Casa,Leve,724,ESPAÑA,Recuperado,7/3/2020 0:00:00,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6,NaN,NaN
9,12/3/2020 0:00:00,10,12/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,36,1,F,Importado,Casa,Leve,724,ESPAÑA,Recuperado,6/3/2020 0:00:00,12/3/2020 0:00:00,21/3/2020 0:00:00,PCR,6,NaN,NaN


In [14]:
df.describe()

,web_date_report,case_id,notification_date,davipola_departament_code,department_name,davipola_municipality_code,municipality_name,age,age_measurement_unit,sex,contagion_type,case_location,state,iso_country_code,country_name,recuperated,beginning_symptoms_date,date_of_diagnosis,date_of_recovery,recovery_type,ethnicity,ethnic_group_name,date_of_death
count,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,2000,684,684,2000,1938,1998,1870,1870,2000,18,137
unique,31,2000,37,27,27,117,117,96,2,2,4,3,3,37,37,3,40,31,114,2,4,7,51
top,8/4/2020 0:00:00,444,28/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,29,1,M,Relacionado,Casa,Leve,724,ESPAÑA,Recuperado,20/3/2020 0:00:00,8/4/2020 0:00:00,5/5/2020 0:00:00,PCR,6,Por definir,4/4/2020 0:00:00
freq,292,1,143,975,975,975,975,58,1995,1004,1004,1863,1863,236,236,1870,141,287,116,1695,1937,9,12


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   web_date_report             2000 non-null   object
 1   case_id                     2000 non-null   object
 2   notification_date           2000 non-null   object
 3   davipola_departament_code   2000 non-null   object
 4   department_name             2000 non-null   object
 5   davipola_municipality_code  2000 non-null   object
 6   municipality_name           2000 non-null   object
 7   age                         2000 non-null   object
 8   age_measurement_unit        2000 non-null   object
 9   sex                         2000 non-null   object
 10  contagion_type              2000 non-null   object
 11  case_location               2000 non-null   object
 12  state                       2000 non-null   object
 13  iso_country_code            684 non-null    obje

Se quitan los nulos y se reemplazan por 0

In [16]:
df = df.fillna(0)

verifying nulls


```
print(df.isnull().values.any())
```
[source](https://towardsdatascience.com/a-complete-guide-to-principal-component-analysis-pca-in-machine-learning-664f34fc3e5a)

After use this function, the result of the code is false.


In [17]:
print(df.isnull().values.any())

False


In [18]:
df.date_of_death


0                      0
1                      0
2                      0
3                      0
4                      0
              ...       
1995                   0
1996                   0
1997                   0
1998    7/4/2020 0:00:00
1999                   0
Name: date_of_death, Length: 2000, dtype: object

In [19]:
df.date_of_recovery

0       13/3/2020 0:00:00
1       19/3/2020 0:00:00
2       15/3/2020 0:00:00
3       26/3/2020 0:00:00
4       23/3/2020 0:00:00
              ...        
1995     9/5/2020 0:00:00
1996     9/5/2020 0:00:00
1997    24/5/2020 0:00:00
1998                    0
1999    11/4/2020 0:00:00
Name: date_of_recovery, Length: 2000, dtype: object

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   web_date_report             2000 non-null   object
 1   case_id                     2000 non-null   object
 2   notification_date           2000 non-null   object
 3   davipola_departament_code   2000 non-null   object
 4   department_name             2000 non-null   object
 5   davipola_municipality_code  2000 non-null   object
 6   municipality_name           2000 non-null   object
 7   age                         2000 non-null   object
 8   age_measurement_unit        2000 non-null   object
 9   sex                         2000 non-null   object
 10  contagion_type              2000 non-null   object
 11  case_location               2000 non-null   object
 12  state                       2000 non-null   object
 13  iso_country_code            2000 non-null   obje

Similarity mesurement (radio basics funciotn)
1.  
*   ML / rbf -| knn graph / 
*   train to know about who would most highly survive
*   KNN graph (find the most near neibough
2.  
*   svm (support vector machine)





Adjency matrix. to build a graph after apply rbf and knn aproach?



In [21]:
df.head(10)

,web_date_report,case_id,notification_date,davipola_departament_code,department_name,davipola_municipality_code,municipality_name,age,age_measurement_unit,sex,contagion_type,case_location,state,iso_country_code,country_name,recuperated,beginning_symptoms_date,date_of_diagnosis,date_of_recovery,recovery_type,ethnicity,ethnic_group_name,date_of_death
0,6/3/2020 0:00:00,1,2/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,19,1,F,Importado,Casa,Leve,380,ITALIA,Recuperado,27/2/2020 0:00:00,6/3/2020 0:00:00,13/3/2020 0:00:00,PCR,6,0,0
1,9/3/2020 0:00:00,2,6/3/2020 0:00:00,76,VALLE,76111,BUGA,34,1,M,Importado,Casa,Leve,724,ESPAÑA,Recuperado,4/3/2020 0:00:00,9/3/2020 0:00:00,19/3/2020 0:00:00,PCR,5,0,0
2,9/3/2020 0:00:00,3,7/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,50,1,F,Importado,Casa,Leve,724,ESPAÑA,Recuperado,29/2/2020 0:00:00,9/3/2020 0:00:00,15/3/2020 0:00:00,PCR,6,0,0
3,11/3/2020 0:00:00,4,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,55,1,M,Relacionado,Casa,Leve,0,0,Recuperado,6/3/2020 0:00:00,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6,0,0
4,11/3/2020 0:00:00,5,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,25,1,M,Relacionado,Casa,Leve,0,0,Recuperado,8/3/2020 0:00:00,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6,0,0
5,11/3/2020 0:00:00,6,10/3/2020 0:00:00,5,ANTIOQUIA,5360,ITAGUI,27,1,F,Relacionado,Casa,Leve,0,0,Recuperado,6/3/2020 0:00:00,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6,0,0
6,11/3/2020 0:00:00,7,8/3/2020 0:00:00,13001,CARTAGENA,13001,CARTAGENA,85,1,F,Importado,Casa,Leve,840,ESTADOS UNIDOS DE AMÉRICA,Recuperado,2/3/2020 0:00:00,11/3/2020 0:00:00,17/3/2020 0:00:00,PCR,6,0,0
7,11/3/2020 0:00:00,8,9/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,22,1,F,Importado,Casa,Leve,724,ESPAÑA,Recuperado,6/3/2020 0:00:00,11/3/2020 0:00:00,21/3/2020 0:00:00,PCR,6,0,0
8,11/3/2020 0:00:00,9,8/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,28,1,F,Importado,Casa,Leve,724,ESPAÑA,Recuperado,7/3/2020 0:00:00,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6,0,0
9,12/3/2020 0:00:00,10,12/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,36,1,F,Importado,Casa,Leve,724,ESPAÑA,Recuperado,6/3/2020 0:00:00,12/3/2020 0:00:00,21/3/2020 0:00:00,PCR,6,0,0


In [22]:
"""
model = tf.keras.Sequential([
  tf.layers.Dense(input_shape=(28,28,3))),
  tf.layers.Dense(100, activation='relu'),
  tf.layers.Dense(100, activation='relu'),
  tf.layers.Dense(10, activation='softmax')
])
"""

"\nmodel = tf.keras.Sequential([\n  tf.layers.Dense(input_shape=(28,28,3))),\n  tf.layers.Dense(100, activation='relu'),\n  tf.layers.Dense(100, activation='relu'),\n  tf.layers.Dense(10, activation='softmax')\n])\n"

In [23]:
for i in df.columns:
  if i.find("date") != -1:
    print(i)
    a = list()
    for j in eval("df."+i):
      #if j = null
      if j != 0:
        a.append(datetime.strptime(str(j), '%d/%m/%Y %H:%M:%S').timestamp())
      else:
        a.append(0)
    df[i] = a

web_date_report
notification_date
beginning_symptoms_date
date_of_diagnosis
date_of_recovery
date_of_death


In [24]:
df.values

array([[1583452800.0, '1', 1583107200.0, ..., '6', 0, 0.0],
       [1583712000.0, '2', 1583452800.0, ..., '5', 0, 0.0],
       [1583712000.0, '3', 1583539200.0, ..., '6', 0, 0.0],
       ...,
       [1586304000.0, '1998', 1585353600.0, ..., '6', 0, 0.0],
       [1586304000.0, '1999', 1585180800.0, ..., '6', 0, 1586217600.0],
       [1586304000.0, '2000', 1585267200.0, ..., '6', 0, 0.0]],
      dtype=object)

1.   Delete departament name && municipality name 
2.   Binarize sex column
3.   Delete country name
4.   Normalize recovery type
5.   Write documentation


In [25]:
df.describe()

,web_date_report,notification_date,beginning_symptoms_date,date_of_diagnosis,date_of_recovery,date_of_death
count,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03,2.000000e+03
mean,1.585609e+09,1.585197e+09,1.535553e+09,1.584023e+09,1.484463e+09,1.087885e+08
std,5.807891e+05,5.252971e+05,2.747219e+08,5.013214e+07,3.915072e+08,4.012747e+08
min,1.583453e+09,1.583107e+09,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.585267e+09,1.584922e+09,1.584230e+09,1.585267e+09,1.586131e+09,0.000000e+00
50%,1.585699e+09,1.585267e+09,1.584662e+09,1.585699e+09,1.586909e+09,0.000000e+00
75%,1.586131e+09,1.585526e+09,1.585008e+09,1.586131e+09,1.588637e+09,0.000000e+00
max,1.586304e+09,1.593821e+09,1.593562e+09,1.586304e+09,1.622938e+09,1.620000e+09


Analisis de componentes principales (APC)

quitar lo que se puede.



desviación estandar de  para analizar causas de muerte

.fit

se realiza conversión a todos los que tengan

In [26]:
#/-////////////////////////////////////////////////////////#
#                                                         /
#              average of recovery time per genre         /
#                                                         /
#/////////////////////////////////////////////////////////#

#////////////////////////////////////////////////////////#
#                                                        /
#     Capitalize Sex column values to avoid errors       /
#                                                        /
#////////////////////////////////////////////////////////#

#///



#////////////////////////////////////////////////////////#
#                                                        /
#  substract betweeen two dates and store it the answer  /
#                                                        /
#////////////////////////////////////////////////////////#

def get_duration_of_illness(x:any):
  time = 0
  z = list()
  count = 0
  for e in x.index:
    # print("position:")
    # print(e)
    # print(x.recuperated[e])
    count += 1
    if x.date_of_recovery[e] != 0:
      x.loc[e, "time_to_recover"] = abs(x.date_of_recovery[e] - x.beginning_symptoms_date[e])
    elif x.date_of_death[e] != 0:
      x.loc[e, "time_to_recover"] = abs(x.date_of_death[e] - x.beginning_symptoms_date[e])
    elif x.date_of_death[e] == 0 and x.date_of_recovery[e] == 0:
      print("NAN")
      x.loc[e, "time_to_recover"] = 0
  
  print("The total amount of data is: "+ str(count))

  return x

      # x.duration_of_illness.append()
      
      # abs((d2 - d1).days)

#////////////////////////////////////////////////////////#
#                                                        /
# Get important Columns to get the main of recovery time /
#                                                        /
#////////////////////////////////////////////////////////#

mean = 0
array_duration_of_illness = df.loc[
                                   :,
                                   [
                                    "sex",
                                    "beginning_symptoms_date",
                                    "date_of_recovery",
                                    "date_of_death",
                                    "recuperated"
                                    ]
                                   ].copy()

                                   #  sex M/ 15 avg to get recover from beginning_symptoms_date. 
                                   #  sex F/ 14 avg.

                                   # Male / Ibague / how long time do they neeed to get recover?
                                   #cound .. avg ... 

                                   ## share tje ML research results -| graph clustering. 

array_dates_women = array_duration_of_illness[array_duration_of_illness["sex"] == 'F']
get_duration_of_illness(array_dates_women)

array_dates_men = array_duration_of_illness[array_duration_of_illness["sex"] == "M"]
get_duration_of_illness(array_dates_men)

print(df.iloc[471])

#////           For alive people
# df.beginning_symptoms_date - df.date_of_recovery 


#////           For death people
# df.beginning_symptoms_date - df.date_of_death 

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


The total amount of data is: 996
The total amount of data is: 1004
web_date_report               1.58509e+09
case_id                               472
notification_date             1.58484e+09
davipola_departament_code              11
department_name                    BOGOTA
davipola_municipality_code          11001
municipality_name                  BOGOTA
age                                    19
age_measurement_unit                    1
sex                                     F
contagion_type                  Importado
case_location                        Casa
state                                Leve
iso_country_code                      276
country_name                     ALEMANIA
recuperated                    Recuperado
beginning_symptoms_date       1.58449e+09
date_of_diagnosis             1.58509e+09
date_of_recovery              1.58812e+09
recovery_type                         PCR
ethnicity                               6
ethnic_group_name                       0
date_of_d

Replacing


In [27]:
#df.replace([{'sex': {'F': 0, 'M': 1}]}) #Female = 0; Male = 1
df.replace(['F','M','Comunitaria','Relacionado','En estudio','Importado','Importado','Casa'],['1','0','0','1','2','3','4','House'])

,web_date_report,case_id,notification_date,davipola_departament_code,department_name,davipola_municipality_code,municipality_name,age,age_measurement_unit,sex,contagion_type,case_location,state,iso_country_code,country_name,recuperated,beginning_symptoms_date,date_of_diagnosis,date_of_recovery,recovery_type,ethnicity,ethnic_group_name,date_of_death
0,1.583453e+09,1,1.583107e+09,11,BOGOTA,11001,BOGOTA,19,1,1,4,House,Leve,380,ITALIA,Recuperado,1.582762e+09,1.583453e+09,1.584058e+09,PCR,6,0,0.000000e+00
1,1.583712e+09,2,1.583453e+09,76,VALLE,76111,BUGA,34,1,0,4,House,Leve,724,ESPAÑA,Recuperado,1.583280e+09,1.583712e+09,1.584576e+09,PCR,5,0,0.000000e+00
2,1.583712e+09,3,1.583539e+09,5,ANTIOQUIA,5001,MEDELLIN,50,1,1,4,House,Leve,724,ESPAÑA,Recuperado,1.582934e+09,1.583712e+09,1.584230e+09,PCR,6,0,0.000000e+00
3,1.583885e+09,4,1.583712e+09,5,ANTIOQUIA,5001,MEDELLIN,55,1,0,1,House,Leve,0,0,Recuperado,1.583453e+09,1.583885e+09,1.585181e+09,PCR,6,0,0.000000e+00
4,1.583885e+09,5,1.583712e+09,5,ANTIOQUIA,5001,MEDELLIN,25,1,0,1,House,Leve,0,0,Recuperado,1.583626e+09,1.583885e+09,1.584922e+09,PCR,6,0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1.586304e+09,1996,1.585181e+09,5,ANTIOQUIA,5001,MEDELLIN,54,1,0,4,House,Leve,724,ESPAÑA,Recuperado,1.584490e+09,1.586304e+09,1.588982e+09,Tiempo,6,0,0.000000e+00
1996,1.586304e+09,1997,1.584835e+09,5,ANTIOQUIA,5088,BELLO,36,1,1,4,House,Leve,388,JAMAICA,Recuperado,1.584144e+09,1.586304e+09,1.588982e+09,PCR,6,0,0.000000e+00
1997,1.586304e+09,1998,1.585354e+09,5,ANTIOQUIA,5001,MEDELLIN,60,1,0,4,House,Leve,840,ESTADOS UNIDOS DE AMÉRICA,Recuperado,1.584922e+09,1.586304e+09,1.590278e+09,Tiempo,6,0,0.000000e+00
1998,1.586304e+09,1999,1.585181e+09,47001,STA MARTA D.E.,47001,SANTA MARTA,65,1,0,1,Fallecido,Fallecido,0,0,Fallecido,1.584922e+09,1.586304e+09,0.000000e+00,0,6,0,1.586218e+09


#Euclidean function



In [28]:
euclidean_function_df = df.iloc[:, lambda df: [2,3,5,8,9,10,11,12,13,15,16,17,18,20,21]].copy()
euclidean_function_df

,notification_date,davipola_departament_code,davipola_municipality_code,age_measurement_unit,sex,contagion_type,case_location,state,iso_country_code,recuperated,beginning_symptoms_date,date_of_diagnosis,date_of_recovery,ethnicity,ethnic_group_name
0,1.583107e+09,11,11001,1,F,Importado,Casa,Leve,380,Recuperado,1.582762e+09,1.583453e+09,1.584058e+09,6,0
1,1.583453e+09,76,76111,1,M,Importado,Casa,Leve,724,Recuperado,1.583280e+09,1.583712e+09,1.584576e+09,5,0
2,1.583539e+09,5,5001,1,F,Importado,Casa,Leve,724,Recuperado,1.582934e+09,1.583712e+09,1.584230e+09,6,0
3,1.583712e+09,5,5001,1,M,Relacionado,Casa,Leve,0,Recuperado,1.583453e+09,1.583885e+09,1.585181e+09,6,0
4,1.583712e+09,5,5001,1,M,Relacionado,Casa,Leve,0,Recuperado,1.583626e+09,1.583885e+09,1.584922e+09,6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1.585181e+09,5,5001,1,M,Importado,Casa,Leve,724,Recuperado,1.584490e+09,1.586304e+09,1.588982e+09,6,0
1996,1.584835e+09,5,5088,1,F,Importado,Casa,Leve,388,Recuperado,1.584144e+09,1.586304e+09,1.588982e+09,6,0
1997,1.585354e+09,5,5001,1,M,Importado,Casa,Leve,840,Recuperado,1.584922e+09,1.586304e+09,1.590278e+09,6,0
1998,1.585181e+09,47001,47001,1,M,Relacionado,Fallecido,Fallecido,0,Fallecido,1.584922e+09,1.586304e+09,0.000000e+00,6,0


#Requesting more than 1000 records.

In [29]:
import os
!pip install sodapy -q
!pip install socrata-py -q

from sodapy import Socrata
from socrata.authorization import Authorization

In [30]:
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

results_df

,fecha_reporte_web,id_de_caso,fecha_de_notificaci_n,departamento,departamento_nom,ciudad_municipio,ciudad_municipio_nom,edad,unidad_medida,sexo,fuente_tipo_contagio,ubicacion,estado,pais_viajo_1_cod,pais_viajo_1_nom,recuperado,fecha_inicio_sintomas,fecha_diagnostico,fecha_recuperado,tipo_recuperacion,per_etn_,nom_grupo_,fecha_muerte
0,6/3/2020 0:00:00,1,2/3/2020 0:00:00,11,BOGOTA,11001,BOGOTA,19,1,F,Importado,Casa,Leve,380,ITALIA,Recuperado,27/2/2020 0:00:00,6/3/2020 0:00:00,13/3/2020 0:00:00,PCR,6,NaN,NaN
1,9/3/2020 0:00:00,2,6/3/2020 0:00:00,76,VALLE,76111,BUGA,34,1,M,Importado,Casa,Leve,724,ESPAÑA,Recuperado,4/3/2020 0:00:00,9/3/2020 0:00:00,19/3/2020 0:00:00,PCR,5,NaN,NaN
2,9/3/2020 0:00:00,3,7/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,50,1,F,Importado,Casa,Leve,724,ESPAÑA,Recuperado,29/2/2020 0:00:00,9/3/2020 0:00:00,15/3/2020 0:00:00,PCR,6,NaN,NaN
3,11/3/2020 0:00:00,4,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,55,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,6/3/2020 0:00:00,11/3/2020 0:00:00,26/3/2020 0:00:00,PCR,6,NaN,NaN
4,11/3/2020 0:00:00,5,9/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,25,1,M,Relacionado,Casa,Leve,NaN,NaN,Recuperado,8/3/2020 0:00:00,11/3/2020 0:00:00,23/3/2020 0:00:00,PCR,6,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,8/4/2020 0:00:00,1996,26/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,54,1,M,Importado,Casa,Leve,724,ESPAÑA,Recuperado,18/3/2020 0:00:00,8/4/2020 0:00:00,9/5/2020 0:00:00,Tiempo,6,NaN,NaN
1996,8/4/2020 0:00:00,1997,22/3/2020 0:00:00,5,ANTIOQUIA,5088,BELLO,36,1,F,Importado,Casa,Leve,388,JAMAICA,Recuperado,14/3/2020 0:00:00,8/4/2020 0:00:00,9/5/2020 0:00:00,PCR,6,NaN,NaN
1997,8/4/2020 0:00:00,1998,28/3/2020 0:00:00,5,ANTIOQUIA,5001,MEDELLIN,60,1,M,Importado,Casa,Leve,840,ESTADOS UNIDOS DE AMÉRICA,Recuperado,23/3/2020 0:00:00,8/4/2020 0:00:00,24/5/2020 0:00:00,Tiempo,6,NaN,NaN
1998,8/4/2020 0:00:00,1999,26/3/2020 0:00:00,47001,STA MARTA D.E.,47001,SANTA MARTA,65,1,M,Relacionado,Fallecido,Fallecido,NaN,NaN,Fallecido,23/3/2020 0:00:00,8/4/2020 0:00:00,NaN,NaN,6,NaN,7/4/2020 0:00:00


In [31]:
results_df.describe

<bound method NDFrame.describe of       fecha_reporte_web id_de_caso  ... nom_grupo_      fecha_muerte
0      6/3/2020 0:00:00          1  ...        NaN               NaN
1      9/3/2020 0:00:00          2  ...        NaN               NaN
2      9/3/2020 0:00:00          3  ...        NaN               NaN
3     11/3/2020 0:00:00          4  ...        NaN               NaN
4     11/3/2020 0:00:00          5  ...        NaN               NaN
...                 ...        ...  ...        ...               ...
1995   8/4/2020 0:00:00       1996  ...        NaN               NaN
1996   8/4/2020 0:00:00       1997  ...        NaN               NaN
1997   8/4/2020 0:00:00       1998  ...        NaN               NaN
1998   8/4/2020 0:00:00       1999  ...        NaN  7/4/2020 0:00:00
1999   8/4/2020 0:00:00       2000  ...        NaN               NaN

[2000 rows x 23 columns]>

In [32]:
results_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   fecha_reporte_web      2000 non-null   object
 1   id_de_caso             2000 non-null   object
 2   fecha_de_notificaci_n  2000 non-null   object
 3   departamento           2000 non-null   object
 4   departamento_nom       2000 non-null   object
 5   ciudad_municipio       2000 non-null   object
 6   ciudad_municipio_nom   2000 non-null   object
 7   edad                   2000 non-null   object
 8   unidad_medida          2000 non-null   object
 9   sexo                   2000 non-null   object
 10  fuente_tipo_contagio   2000 non-null   object
 11  ubicacion              2000 non-null   object
 12  estado                 2000 non-null   object
 13  pais_viajo_1_cod       684 non-null    object
 14  pais_viajo_1_nom       684 non-null    object
 15  recuperado           

#Compromiso

1. Servicio que colecta la data por departamento y la guarda en la base de datos
    * Eliminar registros por departamento y volver a consultarlos y almacenarlos
2. Servicio que recibe por parametro el id del departamento 
3. Servicio que recupera la info por municipio.


#Commitment

1. Service which collect the data from Colombia according to each departament (state)
    * Delete records per departament and request it again
2. Service which get by parameter the departament_id and return the information
3. Service get municipality info


Documentation

Contagion types

Json
```
[
  {"fuente_tipo_contagio":"Comunitaria"}
  ,{"fuente_tipo_contagio":"Relacionado"}
  ,{"fuente_tipo_contagio":"En estudio"}
  ,{"fuente_tipo_contagio":"Importado"}
]
```

request
[location and count](https://datos.gov.co/resource/gt2j-8ykr.json?$select=%20count(id_de_caso),ubicacion&$group=ubicacion)


```
count_id_de_caso	"11015"
ubicacion	"casa"
count_id_de_caso	"4028443"
ubicacion	"Casa"
count_id_de_caso	"7"
ubicacion	"CASA"
count_id_de_caso	"105326"
ubicacion	"Fallecido"
count_id_de_caso	"22865"
ubicacion	"Hospital"
count_id_de_caso	"4963"
ubicacion	"Hospital UCI"
count_id_de_caso	"14575"
ubicacion	"N/A"
```

